## Poker Equity Analyser Demo

This notebook demonstrates the key features of the poker equity calculation system, including hand evaluation, Monte Carlo simulation, and mathematical decision making.


In [ ]:
import sys
sys.path.append('..')

from src.poker_engine import Card, Deck, Rank, Suit
from src.hand_evaluator import HandEvaluator, HandRank
from src.monte_carlo import MonteCarloSimulator
from src.strategy import StrategyCalculator

print("Poker Equity Analyser loaded successfully!")

Poker Equity Analyser loaded successfully!


### 1. Basic Components
#### Creating Cards and Decks

In [3]:
# Create some cards
ace_spades = Card(Rank.ACE, Suit.SPADES)
ace_hearts = Card(Rank.ACE, Suit.HEARTS)
king_clubs = Card(Rank.KING, Suit.CLUBS)

print(f"Single card: {ace_spades}")
print(f"Multiple cards: {ace_spades}, {ace_hearts}, {king_clubs}")

# Create and shuffle a deck
deck = Deck()
print(f"\n{deck}")

# Deal some hands
hands = deck.deal_to_players(2, 2)
for i, hand in enumerate(hands, 1):
    print(f"Player {i}: {hand[0]}, {hand[1]}")

Single card: A♠
Multiple cards: A♠, A♥, K♣

Deck (52 cards): 4♠, 8♥, 8♣, A♥, 7♦, ...
Player 1: 7♠, T♠
Player 2: 2♠, 6♠


### 2. Hand Evaluation
#### Demonstrating All Poker Hand Rankings

In [4]:
def create_hand_examples():
    """Create example hands for each ranking."""
    examples = {
        "High Card": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.KING, Suit.HEARTS),
            Card(Rank.QUEEN, Suit.CLUBS),
            Card(Rank.JACK, Suit.DIAMONDS),
            Card(Rank.NINE, Suit.SPADES)
        ],
        "Pair": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.ACE, Suit.HEARTS),
            Card(Rank.KING, Suit.CLUBS),
            Card(Rank.QUEEN, Suit.DIAMONDS),
            Card(Rank.JACK, Suit.SPADES)
        ],
        "Two Pair": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.ACE, Suit.HEARTS),
            Card(Rank.KING, Suit.CLUBS),
            Card(Rank.KING, Suit.DIAMONDS),
            Card(Rank.QUEEN, Suit.SPADES)
        ],
        "Three of a Kind": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.ACE, Suit.HEARTS),
            Card(Rank.ACE, Suit.CLUBS),
            Card(Rank.KING, Suit.DIAMONDS),
            Card(Rank.QUEEN, Suit.SPADES)
        ],
        "Straight": [
            Card(Rank.FIVE, Suit.SPADES),
            Card(Rank.FOUR, Suit.HEARTS),
            Card(Rank.THREE, Suit.CLUBS),
            Card(Rank.TWO, Suit.DIAMONDS),
            Card(Rank.ACE, Suit.SPADES)
        ],
        "Flush": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.KING, Suit.SPADES),
            Card(Rank.QUEEN, Suit.SPADES),
            Card(Rank.JACK, Suit.SPADES),
            Card(Rank.NINE, Suit.SPADES)
        ],
        "Full House": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.ACE, Suit.HEARTS),
            Card(Rank.ACE, Suit.CLUBS),
            Card(Rank.KING, Suit.DIAMONDS),
            Card(Rank.KING, Suit.SPADES)
        ],
        "Four of a Kind": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.ACE, Suit.HEARTS),
            Card(Rank.ACE, Suit.CLUBS),
            Card(Rank.ACE, Suit.DIAMONDS),
            Card(Rank.KING, Suit.SPADES)
        ],
        "Straight Flush": [
            Card(Rank.NINE, Suit.SPADES),
            Card(Rank.EIGHT, Suit.SPADES),
            Card(Rank.SEVEN, Suit.SPADES),
            Card(Rank.SIX, Suit.SPADES),
            Card(Rank.FIVE, Suit.SPADES)
        ],
        "Royal Flush": [
            Card(Rank.ACE, Suit.SPADES),
            Card(Rank.KING, Suit.SPADES),
            Card(Rank.QUEEN, Suit.SPADES),
            Card(Rank.JACK, Suit.SPADES),
            Card(Rank.TEN, Suit.SPADES)
        ]
    }
    return examples

# Display all hand types
examples = create_hand_examples()
for hand_name, cards in examples.items():
    result = HandEvaluator.evaluate_hand(cards)
    cards_str = ', '.join(str(c) for c in cards[:5])
    print(f"{hand_name:15} | {cards_str:30} | Rank: {result[0].name}")

High Card       | A♠, K♥, Q♣, J♦, 9♠             | Rank: HIGH_CARD
Pair            | A♠, A♥, K♣, Q♦, J♠             | Rank: PAIR
Two Pair        | A♠, A♥, K♣, K♦, Q♠             | Rank: TWO_PAIR
Three of a Kind | A♠, A♥, A♣, K♦, Q♠             | Rank: THREE_OF_A_KIND
Straight        | 5♠, 4♥, 3♣, 2♦, A♠             | Rank: STRAIGHT
Flush           | A♠, K♠, Q♠, J♠, 9♠             | Rank: FLUSH
Full House      | A♠, A♥, A♣, K♦, K♠             | Rank: FULL_HOUSE
Four of a Kind  | A♠, A♥, A♣, A♦, K♠             | Rank: FOUR_OF_A_KIND
Straight Flush  | 9♠, 8♠, 7♠, 6♠, 5♠             | Rank: STRAIGHT_FLUSH
Royal Flush     | A♠, K♠, Q♠, J♠, T♠             | Rank: ROYAL_FLUSH


### 3. Classic Poker Matchups
#### Pre-flop Equity in Common Scenarios

In [7]:
def run_matchup(hand1_cards, hand2_cards, description, iterations=10000):
    """Run a heads-up matchup and display results."""
    print(f"\n{description}")
    print(f"Hand 1: {hand1_cards[0]}, {hand1_cards[1]}")
    print(f"Hand 2: {hand2_cards[0]}, {hand2_cards[1]}")
    
    # Calculate equity for hand 1
    results = MonteCarloSimulator.calculate_equity(
        hand1_cards,
        community_cards=None,
        num_opponents=1,
        iterations=iterations
    )
    
    print(f"Hand 1 Equity vs Random: {results['win']:.1%}")
    print("-" * 40)

# Classic matchups
print("="*50)
print("CLASSIC POKER MATCHUPS")
print("="*50)

# Pocket Aces vs Kings (AA ~80% favorite)
aa = [Card(Rank.ACE, Suit.SPADES), Card(Rank.ACE, Suit.HEARTS)]
kk = [Card(Rank.KING, Suit.CLUBS), Card(Rank.KING, Suit.DIAMONDS)]
run_matchup(aa, kk, "AA vs KK - The Classic Cooler")

# AK vs QQ (Coin flip ~50%)
ak = [Card(Rank.ACE, Suit.SPADES), Card(Rank.KING, Suit.SPADES)]
qq = [Card(Rank.QUEEN, Suit.CLUBS), Card(Rank.QUEEN, Suit.DIAMONDS)]
run_matchup(ak, qq, "AKs vs QQ - The Race")

# AA vs 72o (Maximum domination)
aa = [Card(Rank.ACE, Suit.SPADES), Card(Rank.ACE, Suit.HEARTS)]
worst = [Card(Rank.TWO, Suit.CLUBS), Card(Rank.SEVEN, Suit.DIAMONDS)]
run_matchup(aa, worst, "AA vs 72o - Best vs Worst")

CLASSIC POKER MATCHUPS

AA vs KK - The Classic Cooler
Hand 1: A♠, A♥
Hand 2: K♣, K♦
Hand 1 Equity vs Random: 84.6%
----------------------------------------

AKs vs QQ - The Race
Hand 1: A♠, K♠
Hand 2: Q♣, Q♦
Hand 1 Equity vs Random: 66.9%
----------------------------------------

AA vs 72o - Best vs Worst
Hand 1: A♠, A♥
Hand 2: 2♣, 7♦
Hand 1 Equity vs Random: 84.5%
----------------------------------------


### 4. Monte Carlo Performance Analysis
#### Accuracy vs Speed Tradeoff

In [9]:
pocket_aces = [Card(Rank.ACE, Suit.SPADES), Card(Rank.ACE, Suit.HEARTS)]
iteration_counts = [100, 500, 1000, 5000, 10000, 50000]
results_data = []

print("Analyzing accuracy and performance across different iteration counts...")
print("="*60)
print(f"{'Iterations':<12} {'Win %':<10} {'Time (s)':<10} {'Speed (iter/s)':<15}")
print("-"*60)

for iterations in iteration_counts:
    results = MonteCarloSimulator.calculate_equity(
        pocket_aces,
        community_cards=None,
        num_opponents=1,
        iterations=iterations,
        measure_time=True
    )
    
    print(f"{iterations:<12} {results['win']*100:<10.2f} {results['time_seconds']:<10.4f} "
          f"{results['iterations_per_second']:<15.0f}")
    
    results_data.append({
        'iterations': iterations,
        'win_pct': results['win'] * 100,
        'time': results['time_seconds']
    })

print("-"*60)
print("Note: AA should win ~85% vs random opponent")
print("Higher iterations = more accurate but slower")

Analyzing accuracy and performance across different iteration counts...
Iterations   Win %      Time (s)   Speed (iter/s) 
------------------------------------------------------------
100          82.00      0.0044     22886          
500          84.80      0.0229     21823          
1000         83.40      0.0430     23281          
5000         85.14      0.2146     23297          
10000        84.39      0.4302     23245          
50000        84.96      2.1422     23341          
------------------------------------------------------------
Note: AA should win ~85% vs random opponent
Higher iterations = more accurate but slower


### 5. Real Poker Decision Scenarios
#### Using Pot Odds and EV to Make Optimal Decisions

In [10]:
print("="*60)
print("SCENARIO 1: Flush Draw on the Turn")
print("="*60)

# Setup
hero_cards = [Card(Rank.ACE, Suit.SPADES), Card(Rank.KING, Suit.SPADES)]
board = [
    Card(Rank.QUEEN, Suit.SPADES),
    Card(Rank.JACK, Suit.HEARTS),
    Card(Rank.FIVE, Suit.SPADES),
    Card(Rank.TWO, Suit.CLUBS)
]

pot = 200
bet = 50

print(f"Your hand: {hero_cards[0]}, {hero_cards[1]}")
print(f"Board: {' '.join(str(c) for c in board)}")
print(f"\nPot: ${pot}")
print(f"Opponent bets: ${bet}")
print("\nYou have:")
print("- Nut flush draw (9 outs)")
print("- Two overcards (potentially 6 more outs)")

# Get decision
decision = StrategyCalculator.get_decision(
    hero_cards,
    board,
    pot_size=pot,
    call_amount=bet,
    iterations=10000
)

print(f"\n--- Analysis ---")
print(f"Your equity: {decision['equity']:.1%}")
print(f"Pot odds: {decision['pot_odds']:.1%}")
print(f"Expected value: ${decision['ev']:.2f}")
print(f"Decision: {decision['action'].upper()}")
print(f"Profitable: {'Yes' if decision['profitable'] else 'No'}")

SCENARIO 1: Flush Draw on the Turn
Your hand: A♠, K♠
Board: Q♠ J♥ 5♠ 2♣

Pot: $200
Opponent bets: $50

You have:
- Nut flush draw (9 outs)
- Two overcards (potentially 6 more outs)

--- Analysis ---
Your equity: 62.9%
Pot odds: 20.0%
Expected value: $138.79
Decision: CALL
Profitable: Yes


In [11]:
print("="*60)
print("SCENARIO 2: Top Pair, Good Kicker on River")
print("="*60)

hero_cards = [Card(Rank.ACE, Suit.HEARTS), Card(Rank.QUEEN, Suit.DIAMONDS)]
board = [
    Card(Rank.ACE, Suit.CLUBS),
    Card(Rank.JACK, Suit.HEARTS),
    Card(Rank.SEVEN, Suit.SPADES),
    Card(Rank.THREE, Suit.CLUBS),
    Card(Rank.TWO, Suit.DIAMONDS)
]

pot = 300
bet = 150

print(f"Your hand: {hero_cards[0]}, {hero_cards[1]}")
print(f"Board: {' '.join(str(c) for c in board)}")
print(f"\nPot: ${pot}")
print(f"Opponent bets: ${bet} (half pot)")
print("\nYou have: Top pair, good kicker")

decision = StrategyCalculator.get_decision(
    hero_cards,
    board,
    pot_size=pot,
    call_amount=bet,
    iterations=10000
)

print(f"\n--- Analysis ---")
print(f"Your equity: {decision['equity']:.1%}")
print(f"Pot odds: {decision['pot_odds']:.1%}")
print(f"Expected value: ${decision['ev']:.2f}")
print(f"Decision: {decision['action'].upper()}")
print(f"Profitable: {'Yes' if decision['profitable'] else 'No'}")

SCENARIO 2: Top Pair, Good Kicker on River
Your hand: A♥, Q♦
Board: A♣ J♥ 7♠ 3♣ 2♦

Pot: $300
Opponent bets: $150 (half pot)

You have: Top pair, good kicker

--- Analysis ---
Your equity: 88.4%
Pot odds: 33.3%
Expected value: $380.37
Decision: CALL
Profitable: Yes


In [14]:
print("="*60)
print("SCENARIO 3: Free Card Opportunity")
print("="*60)

hero_cards = [Card(Rank.JACK, Suit.HEARTS), Card(Rank.TEN, Suit.HEARTS)]
board = [
    Card(Rank.NINE, Suit.CLUBS),
    Card(Rank.EIGHT, Suit.DIAMONDS),
    Card(Rank.FOUR, Suit.SPADES)
]

pot = 100
bet = 0  # Opponent checks

print(f"Your hand: {hero_cards[0]}, {hero_cards[1]}")
print(f"Board: {' '.join(str(c) for c in board)}")
print(f"\nPot: ${pot}")
print("Opponent checks")
print("\nYou have: Open-ended straight draw")

decision = StrategyCalculator.get_decision(
    hero_cards,
    board,
    pot_size=pot,
    call_amount=bet,
    iterations=10000
)

print("\n--- Analysis ---")
print(f"Your equity: {decision['equity']:.1%}")
print(f"Decision: {decision['action'].upper()}")

SCENARIO 3: Free Card Opportunity
Your hand: J♥, T♥
Board: 9♣ 8♦ 4♠

Pot: $100
Opponent checks

You have: Open-ended straight draw

--- Analysis ---
Your equity: 54.9%
Decision: CHECK


### 6. Interactive Decision Helper
#### Create Your Own Scenarios

In [15]:
def analyze_hand(my_cards_str, board_str=None, pot=100, bet=25):
    """
    Analyze a poker situation.
    
    Parameters:
    my_cards_str : str
        Your cards, e.g., "AS KH" for Ace of Spades, King of Hearts
    board_str : str, optional
        Community cards, e.g., "QS JH 5C"
    pot : float
        Current pot size
    bet : float
        Amount to call
    """
    # Parse card strings
    rank_map = {'A': Rank.ACE, 'K': Rank.KING, 'Q': Rank.QUEEN, 'J': Rank.JACK,
                'T': Rank.TEN, '9': Rank.NINE, '8': Rank.EIGHT, '7': Rank.SEVEN,
                '6': Rank.SIX, '5': Rank.FIVE, '4': Rank.FOUR, '3': Rank.THREE,
                '2': Rank.TWO}
    suit_map = {'S': Suit.SPADES, 'H': Suit.HEARTS, 'D': Suit.DIAMONDS, 'C': Suit.CLUBS}
    
    # Parse hole cards
    my_cards = []
    for card in my_cards_str.split():
        rank = rank_map[card[0]]
        suit = suit_map[card[1]]
        my_cards.append(Card(rank, suit))
    
    # Parse board if provided
    board = []
    if board_str:
        for card in board_str.split():
            rank = rank_map[card[0]]
            suit = suit_map[card[1]]
            board.append(Card(rank, suit))
    
    # Get decision
    decision = StrategyCalculator.get_decision(
        my_cards,
        board if board else None,
        pot_size=pot,
        call_amount=bet,
        iterations=10000
    )
    
    # Display results
    print(f"Your hand: {', '.join(str(c) for c in my_cards)}")
    if board:
        print(f"Board: {', '.join(str(c) for c in board)}")
    print(f"Pot: ${pot}, Bet: ${bet}")
    print(f"\nEquity: {decision['equity']:.1%}")
    print(f"Pot Odds: {decision['pot_odds']:.1%}")
    print(f"EV: ${decision['ev']:.2f}")
    print(f"Decision: {decision['action'].upper()}")
    print(f"Profitable: {'Yes' if decision['profitable'] else 'No'}")

# Example usage
print("Example Usage:")
print("-" * 40)
analyze_hand("AH KH", "QH JH 5S 2C", pot=200, bet=50)

Example Usage:
----------------------------------------
Your hand: A♥, K♥
Board: Q♥, J♥, 5♠, 2♣
Pot: $200, Bet: $50

Equity: 63.7%
Pot Odds: 20.0%
EV: $141.13
Decision: CALL
Profitable: Yes


### 7. Key Takeaways

#### Mathematical Poker Principles:
1. **Pot Odds**: The price the pot is offering you on a call
2. **Equity**: Your probability of winning the hand
3. **Expected Value**: The average amount you win or lose with a decision

#### Decision Framework:
- **Call** when your equity exceeds pot odds (positive EV)
- **Fold** when your equity is below pot odds (negative EV)
- **Check** whenever it's free (can't fold when there's no bet!)

#### Performance Notes:
- 1,000 iterations: Fast (~0.05s), reasonable accuracy (±2%)
- 10,000 iterations: Balanced (~0.5s), good accuracy (±0.5%)
- 100,000 iterations: Slow (~5s), excellent accuracy (±0.1%)


### End of Demo

This poker equity analyser provides mathematically optimal decisions based on:
- Hand strength evaluation
- Monte Carlo simulation for equity calculation
- Pot odds and expected value analysis

For more information, see the project documentation on GitHub.